In [ ]:
# !pip install -q langchain-openai langchain-community langchain-core requests duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00


In [1]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
import requests

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [3]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=3e3e0560f92ae935694a2717efccffb4&query={city}'

  response = requests.get(url)

  return response.json()

In [4]:
llm = ChatGroq(model='gemma2-9b-it')

In [5]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [6]:
# Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt


c:\Users\gupta\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
# Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
                        llm=llm,
                        tools=[search_tool, get_weather_data],
                        prompt=prompt
                        )

In [ ]:
# Wrap it with AgentExecutor
agent_executor = AgentExecutor(
                            agent=agent,
                            tools=[search_tool, get_weather_data],
                            verbose=True
                            )

In [13]:
# Invoke
response = agent_executor.invoke({"input": "Find the capital of Chhattisgarh, then find it's current weather condition"})
print(response)



> Entering new AgentExecutor chain...
Thought: I need to find the capital of Chhattisgarh first, then I can get its weather. 
Action: duckduckgo_search
Action Input: capital of chattisgarh10 Jul 2025 — Raipur, the capital of Chhattisgarh , is one of the fastest-developing cities in India. Atal Nagar (Formerly Naya Raipur) is the new planned city ... 23 Aug 2025 — Raipur is the capital of Chhattisgarh , and houses the Chhattisgarh Vidhan Sabha (Legislative Assembly) and the secretariat. The Chhattisgarh High Court, ... 24 Jun 2025 — The capital of Chhattisgarh is Raipur . Raipur is the main political, economic, and cultural centre of the state. It was declared the capital when Chhattisgarh ... During this period Raipur became the capital of Chattisgarh instead of Ratanpur. ... Inspite of this the capital city of Chattisgarh - Raipur is one ... The state of Chattisgarh in India was formed on 1st Nov 2000, when sixteen Chhattisgarhi-speaking southeastern districts of Madhya Pradesh were

In [10]:
response['output']

'The capital of Madhya Pradesh is Bhopal. The current weather condition in Bhopal is Haze with a temperature of 24 degrees Celsius.'